In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import os
import holidays
import datetime as dt
import networkx as nx
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import glob
os.getcwd()
pd.set_option('display.max_columns', 70)

In [2]:
#For NON commercial stop
### PLEASE CHANGE THE FOLDER PATH
non_comstop = pd.read_parquet("~/Downloads/tjj_allptcars")
non_comstop = non_comstop[(non_comstop["REF_REPERAGE_NO"] > 0) & (non_comstop["COMMERCIAL_STOP_NATURE"] == 0)] #Filter for non comercial stops
hours_range = [5, 6, 7, 8, 9, 10, 15, 16, 17, 18, 19, 20]
non_comstop = non_comstop[non_comstop['THR_ARR'].dt.hour.isin(hours_range)]
non_comstop['TRAIN_DEPARTURE_DATE'] = pd.to_datetime(non_comstop['TRAIN_DEPARTURE_DATE'], format='%Y-%m-%d')
non_comstop

,REF_TOD_TRAIN_ID,TRAIN_PREFIX,TRAIN_NUMBER,TRAIN_DEPARTURE_DATE,REF_PTCAR_NO,PTCAR_ORD_NO,THR_ARR,THR_DEP,HR_ARR,HR_DEP,EH_ARR,EH_DEP,REF_REPERAGE_NO,DEPARTURE_AT_DT_ORIGIN,ARRIVAL_AT_DT_ORIGIN,COMMERCIAL_STOP_NATURE,DEPARTURE_REF_LINE_SECTION_NO,LINE,IDENTIFICATOR
3,30430608,None,2835,2022-06-05,518,133,2022-06-05 15:09:00,2022-06-05 15:09:00,2022-06-05 15:09:01,2022-06-05 15:09:01,1,1,1,2022-06-05 15:09:00,2022-06-05 15:09:00,0,1948,50.0,50A
4,31239066,None,2169,2022-10-17,883,104,2022-10-17 19:45:00,2022-10-17 19:45:00,2022-10-17 19:44:59,2022-10-17 19:44:59,0,0,1,2022-10-17 19:45:00,2022-10-17 19:45:00,0,1439,27.0,27B
8,30726256,None,2437,2022-07-26,24,116,2022-07-26 16:09:00,2022-07-26 16:09:00,2022-07-26 16:12:22,2022-07-26 16:12:22,202,202,1,2022-07-26 16:09:00,2022-07-26 16:09:00,0,1101,125.0,125
10,30950039,None,936,2022-09-01,535,138,2022-09-01 15:36:00,2022-09-01 15:36:00,2022-09-01 15:39:24,2022-09-01 15:39:24,204,204,1,2022-09-01 15:36:00,2022-09-01 15:36:00,0,1476,118.0,118
11,30452214,None,3006,2022-06-09,1840,102,2022-06-09 06:51:00,2022-06-09 06:51:00,2022-06-09 06:51:00,2022-06-09 06:51:00,0,0,1,2022-06-09 06:51:00,2022-06-09 06:51:00,0,1410,59.0,59/1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36044818,32466903,None,5778,2023-05-03,1982,107,2023-05-03 08:15:00,2023-05-03 08:15:00,2023-05-03 08:15:32,2023-05-03 08:15:32,32,32,1,2023-05-03 08:15:00,2023-05-03 08:15:00,0,1773,162.0,162
36044823,32466903,None,5778,2023-05-03,913,112,2023-05-03 08:30:00,2023-05-03 08:30:00,2023-05-03 08:31:26,2023-05-03 08:31:26,86,86,1,2023-05-03 08:30:00,2023-05-03 08:30:00,0,1526,162.0,162
36044840,32464007,None,3615,2023-05-03,1638,102,2023-05-03 17:16:00,2023-05-03 17:16:00,2023-05-03 17:17:14,2023-05-03 17:17:14,74,74,1,2023-05-03 17:16:00,2023-05-03 17:16:00,0,594,50.0,50
36044841,32464007,None,3615,2023-05-03,1597,103,2023-05-03 17:16:00,2023-05-03 17:16:00,2023-05-03 17:17:45,2023-05-03 17:17:45,105,105,1,2023-05-03 17:16:00,2023-05-03 17:16:00,0,424,50.0,50


In [3]:
PTCAR = pd.read_csv("./data/PTCAR.csv",sep=";")
non_comstop_ptcar = non_comstop.merge(PTCAR,on='REF_PTCAR_NO',how = 'left')
non_comstop_ptcar = non_comstop_ptcar[non_comstop_ptcar['LONG_NAME_NL'].notna()]
non_comstop_ptcar = non_comstop_ptcar[non_comstop_ptcar['THR_ARR'] != '1900-01-01 00:00:00'].reset_index()
#Any trains arrive within THR_ARR_3min and THR_ARR are behind the the target train
non_comstop_ptcar['THR_ARR_3min'] = non_comstop_ptcar['THR_ARR'] + timedelta(minutes=3)
non_comstop_ptcar = non_comstop_ptcar[['REF_TOD_TRAIN_ID','THR_ARR','THR_ARR_3min','LONG_NAME_NL','TRAIN_DEPARTURE_DATE']]
non_comstop_ptcar = non_comstop_ptcar.reset_index()
non_comstop_ptcar = non_comstop_ptcar.drop(columns='index')
non_comstop_ptcar

,REF_TOD_TRAIN_ID,THR_ARR,THR_ARR_3min,LONG_NAME_NL,TRAIN_DEPARTURE_DATE
0,30430608,2022-06-05 15:09:00,2022-06-05 15:12:00,HANSBEKE,2022-06-05
1,31239066,2022-10-17 19:45:00,2022-10-17 19:48:00,MUIZEN-ROOSTER T,2022-10-17
2,30726256,2022-07-26 16:09:00,2022-07-26 16:12:00,AMPSIN,2022-07-26
3,30950039,2022-09-01 15:36:00,2022-09-01 15:39:00,HAVRE,2022-09-01
4,30452214,2022-06-09 06:51:00,2022-06-09 06:54:00,Y.ARENDSTRAAT,2022-06-09
...,...,...,...,...,...
13335094,32466903,2023-05-03 08:15:00,2023-05-03 08:18:00,BOIS D'AUSSE,2023-05-03
13335095,32466903,2023-05-03 08:30:00,2023-05-03 08:33:00,NAMUR-HERBATTE,2023-05-03
13335096,32464007,2023-05-03 17:16:00,2023-05-03 17:19:00,Y.WEST DRIEHOEK LEDEBERG,2023-05-03
13335097,32464007,2023-05-03 17:16:00,2023-05-03 17:19:00,Y.OOST DRIEHOEK LEDEBERG,2023-05-03


In [4]:
non_comstop_ptcar.dtypes

REF_TOD_TRAIN_ID                 int64
THR_ARR                 datetime64[ns]
THR_ARR_3min            datetime64[ns]
LONG_NAME_NL                    object
TRAIN_DEPARTURE_DATE    datetime64[ns]
dtype: object

In [7]:
### PLEASE CHANGE THE FOLDER PATH
folder_path = '~/Downloads/all_noncomstop_adj_matrices'
non_comstop_ptcar = non_comstop_ptcar.sort_values(by = 'REF_TOD_TRAIN_ID')
unique_dates = non_comstop_ptcar['TRAIN_DEPARTURE_DATE'].unique()
grouped_data = []

for date in unique_dates:
    data_date = non_comstop_ptcar[non_comstop_ptcar['TRAIN_DEPARTURE_DATE'] == date]
    grouped = data_date.groupby(['LONG_NAME_NL'])
    for name, group in grouped:
        group = group.drop_duplicates(subset=['REF_TOD_TRAIN_ID','LONG_NAME_NL'],keep = 'last')
        ref_ids = group['REF_TOD_TRAIN_ID'].values
        mask = (ref_ids[:, np.newaxis] != ref_ids) & \
               (group['THR_ARR'].values[:, np.newaxis] >= group['THR_ARR'].values) & \
               (group['THR_ARR'].values[:, np.newaxis] <= group['THR_ARR_3min'].values)
        ohe = pd.get_dummies(group['REF_TOD_TRAIN_ID'])
        np.fill_diagonal(ohe.values, 0)
        group_ohe = pd.concat([group, ohe], axis=1)
        #Replace the mask with the corresponding train ID
        group_ohe.iloc[:, 5:] = np.where(mask, 1, group_ohe.iloc[:, 5:])
        
        # Create the filename based on the day
        day = np.datetime_as_string(np.datetime64(date), unit='D') #day = np.datetime_as_string(date, unit='D')
        name_with_underscores = "-".join(str(x) for x in name).replace(" ", "-").replace('/', '').replace(':', '') #name_with_underscores = name.replace(" ", "-").replace('/', '').replace(':', '')
        filename = f"{day}_{name_with_underscores}.csv"  # Using 'LONG_NAME_NL' and day in the filename
        #Save the group dataframe as a CSV file in the specified folder
        group_ohe.to_csv(os.path.join(folder_path, filename), index=False)

KeyboardInterrupt: 

In [8]:
# Step 1: Create an empty dictionary
ref_tod_train_totals = {}

# Step 2: Iterate over all files in the folder
folder_path = '/Users/miek/Downloads/all_noncomstop_adj_matrices'  # Replace with the actual path to your folder
file_count = 0
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):  # Assuming all files in the folder are CSV files
        file_count += 1
        file_path = os.path.join(folder_path, filename)

        # Step 3: Load the data into a DataFrame
        df = pd.read_csv(file_path)

        # Step 4: Update the total num_ahead for each REF_TOD_TRAIN_ID
        for _, row in df.iterrows():
            ref_tod_train_id = row["REF_TOD_TRAIN_ID"]
            num_ahead = row.iloc[5:].sum()

            if ref_tod_train_id in ref_tod_train_totals:
                ref_tod_train_totals[ref_tod_train_id] += num_ahead
            else:
                ref_tod_train_totals[ref_tod_train_id] = num_ahead

        # Step 5: Print the current file being processed
        if file_count % 100 == 0:
            print(f"Processing file {file_count}/{len(os.listdir(folder_path))}: {filename}")

# Step 6: Create a new DataFrame from the dictionary data
result_df = pd.DataFrame.from_dict(ref_tod_train_totals, orient="index", columns=["Total_Num_Ahead"])

# Step 6a: Reset the index and rename the column
result_df.reset_index(inplace=True)
result_df.rename(columns={"index": "REF_TOD_TRAIN_ID"}, inplace=True)

# Step 7: Save the resulting DataFrame to a file if needed
result_df.to_csv("num_ahead_noncommercial_summed.csv", index=False)  # Replace "result.csv" with your desired file path

Processing file 100/228413: 2022-08-06_Y.DUDZELE.csv
Processing file 200/228413: 2022-11-27_Y.ZAVENTEM.csv
Processing file 300/228413: 2022-10-30_LIJN-50A-SEINEN-394.csv
Processing file 400/228413: 2023-01-04_MERCHTEM.csv
Processing file 500/228413: 2022-10-22_Y.DIJLEBRUG.csv
Processing file 600/228413: 2022-12-31_FRAIPONT.csv
Processing file 700/228413: 2022-10-14_ANZEGEM.csv
Processing file 800/228413: 2022-08-27_JEMEPPE-SUR-MEUSE.csv
Processing file 900/228413: 2023-02-17_NINOVE-WIJKSPOOR.csv
Processing file 1000/228413: 2022-11-18_VERVIERS-PALAIS.csv
Processing file 1100/228413: 2022-06-29_NATOYE.csv
Processing file 1200/228413: 2022-11-08_MARCHE-LES-DAMES.csv
Processing file 1300/228413: 2023-04-19_GOUVY-FRONTIERE.csv
Processing file 1400/228413: 2022-12-06_Y.LUCHTBAL.csv
Processing file 1500/228413: 2022-10-28_JABBEKE.csv
Processing file 1600/228413: 2022-12-04_GRENS-BRUSSEL-VLAANDEREN-L25N.csv
Processing file 1700/228413: 2022-08-10_NAMECHE.csv
Processing file 1800/228413: 2023-

In [6]:
#folder_path = '/Volumes/Tuyen Nguyen (Henry)/Traindynamiccon'
non_comstop_ptcar = non_comstop_ptcar.sort_values(by = 'REF_TOD_TRAIN_ID')
grouped_data = []

grouped = non_comstop_ptcar.groupby(['LONG_NAME_NL'])
group_count = len(grouped)
current_group = 0

for name, group in grouped:
    current_group += 1

    # Print the progress message
    print(f"Processing group {current_group}/{group_count}: {name}")

    group = group.drop_duplicates(subset=['REF_TOD_TRAIN_ID','LONG_NAME_NL'],keep = 'last')
    ref_ids = group['REF_TOD_TRAIN_ID'].values
    mask = (ref_ids[:, np.newaxis] != ref_ids) & \
    (group['THR_ARR'].values[:, np.newaxis] >= group['THR_ARR'].values) & \
    (group['THR_ARR'].values[:, np.newaxis] <= group['THR_ARR_3min'].values)
    ohe = pd.get_dummies(group['REF_TOD_TRAIN_ID'])
    np.fill_diagonal(ohe.values, 0)
    group_ohe = pd.concat([group, ohe], axis=1)
    group_ohe.iloc[:, 5:] = np.where(mask, 1, group_ohe.iloc[:, 5:])
    grouped_data.append(group_ohe)
non_comstop_count = pd.concat(grouped_data, ignore_index=True)
non_comstop_count

Processing group 1/931: ('AALST',)
Processing group 2/931: ('AALST-OOST',)
Processing group 3/931: ('AALST-ROOSTER OOST',)
Processing group 4/931: ('AALTER',)
Processing group 5/931: ('AARSCHOT',)
Processing group 6/931: ('AARSCHOT-ROOSTER NOORD',)
Processing group 7/931: ('AARSELE',)
Processing group 8/931: ('ACREN',)
Processing group 9/931: ('AIGREMONT-BLOCK 8.7',)
Processing group 10/931: ('AISEAU',)
Processing group 11/931: ('ALKEN',)
Processing group 12/931: ('AMAY',)
Processing group 13/931: ('AMPSIN',)
Processing group 14/931: ('ANDERLECHT',)
Processing group 15/931: ('ANDERLECHT-ROOSTER OOST',)


: 

: 